# Maxwell Snodgrass, Brendan Miller, Rongtian Dong

## Location Weights

For the project, we chose three locations with similar significance. They are Eastern regions for the United States and we split them into 3 locations. New England, Middle Atlantic, and South Atlantic. To balance our weights between each location, we assigned a weight to each of them based off of their current population:

#### New England Population: 14 Million
#### Middle Atlantic Population: 60 Million
#### South Atlantic Population: 62 Million

* New England Weight = .10
* Middle Atlantic Weight = .44
* South Atlantic Weight = .46


## Item Weights

The items that we chose for this project represents college students or graduate students. For each location we measure food and gas, which are neccessities for any student. Then, we study two unique products for each location that relate to students. Many students who live off campus need to deal with fuel/utility bills and/or energy bills for their house. Also, many students tend to eat on the go, and therefore we study the price of food away from home. Apparrel, nonalcaholic beverages, and recreation could also be potential leisure products for students and are worth studying as well. 

New England
* Food (CUUR0110SAF1) = .50
* Gas (CUUR0110SS47014) = .25
* Fuels/Utilities (CUUR0110SAH) = .15
* Apparrel (CUUR0110SAA) = .10

Middle Atlantic
* Food (CUUR0120SAF1) = .45
* Gas (CUUR0120SS47014) = .35
* Nonalcaholic Beverages (CUUR0120SAF114) = .05
* Food Away From Home (CUUR0120SEFV) = .15

South Atlantic
* Food (CUUR0350SAF1) = .40
* Gas (CUUR0350SS47014) = .30
* Energy Services (CUUR0350SEHF) = .20
* Recreation (CUUR0350SAR) = .10

## Total Weights

The formula below represents our weighted aggregate CPI that we will be studying. Each variable is a combination of an item, and the location we used for that item. Our weights were based off the region's population and our assigned weights representing the importance of each item in each location. 

* Formula = .432(Food-NE/MA/SA) + .317(Gas-NE/MA/SA) + .015(Fuels/Utilities-NE) + .01(Apparrel-NE) + .022(Nonalcaholic Beverages-MA) + .066(Food Away From Home-MA) + .092(Energy Services-SA) + .046(Recreation-SA)
             


In [1]:
##All of the packages you will need, make sure to get your BLS key and enshroud it in another file
import requests
import pandas as pd
import json
import os
os.environ
!dir *.py
%run APIkeys.py
key = os.environ.get('BLS_API_key')

 Volume in drive C is OS
 Volume Serial Number is A893-6B2B

 Directory of C:\Users\MaxBo\ECON1923

09/20/2022  04:51 PM               102 APIkeys.py
               1 File(s)            102 bytes
               0 Dir(s)  352,222,543,872 bytes free
API keys loaded


In [10]:
#This main function will create our dataframe and check to see if our datasets exist

def multiSeriesV4(varList,myKey):
    # Input: varList = a list of strings containing the series names
    # Input: myKey =  a string containing your BLS API key
    # Output: new_df = a data frame containing the data collected
    
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    parameters = {
        "seriesid":varList,
        "startyear":"2018", 
        "endyear":"2022",
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey": myKey 
     }

    data = json.dumps(parameters) #this converts the Python dictionary into a JSON format
    
    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    n = len(varList) #number of series requested
    
    new_df = pd.DataFrame(columns=['year', 'period'])
    for item in range(n):
        l = len(json_data['Results']['series'][item]['data']) #length of the list
        if l == 0:
            print('Series ',varList[item],' does not exist')
        else:
            
            print('Series ',varList[item],' exists with ',l,' observations')
            d = json_data['Results']['series'][item]['data']
            current_df = pd.DataFrame(data=d)
            current_df = current_df[['year','period','value']].astype({'value': 'float64'})
            current_df = current_df.rename(columns = {'value':varList[item]}) 
            
            new_df = new_df.merge(current_df, on = ['year','period'],how='outer')
    
    return new_df

In [3]:
##Creating our basket of items

basket1 = ['CUUR0000SA0','CUUR0110SAF1','CUUR0110SS47014','CUUR0110SAH','CUUR0110SAA','CUUR0120SAF1',
'CUUR0120SS47014','CUUR0120SAF114','CUUR0120SEFV','CUUR0350SAF1','CUUR0350SS47014','CUUR0350SEHF','CUUR0350SAR']

In [4]:
#Creating our data frame and checking to see if our data sets exist

cpitable = multiSeriesV4(basket1,key)

Series  CUUR0000SA0  exists with  57  observations
Series  CUUR0110SAF1  exists with  57  observations
Series  CUUR0110SS47014  exists with  57  observations
Series  CUUR0110SAH  exists with  57  observations
Series  CUUR0110SAA  exists with  57  observations
Series  CUUR0120SAF1  exists with  57  observations
Series  CUUR0120SS47014  exists with  57  observations
Series  CUUR0120SAF114  exists with  57  observations
Series  CUUR0120SEFV  exists with  57  observations
Series  CUUR0350SAF1  exists with  57  observations
Series  CUUR0350SS47014  exists with  57  observations
Series  CUUR0350SEHF  exists with  57  observations
Series  CUUR0350SAR  exists with  57  observations


The original function with start point of 2017 revealed that our data was missing observations for 11/12 months of 2017. Therefore we had to change our function to have a start date of 2018 instead.

In [6]:
#pd.set_option("display.max_columns", None, "display.max_rows", 8)          #These two lines of code allow us to toggle
#pd.set_option("display.max_columns", None, "display.max_rows", None)       #between full table view and a limited table view

cpitable

,year,period,CUUR0000SA0,CUUR0110SAF1,CUUR0110SS47014,CUUR0110SAH,CUUR0110SAA,CUUR0120SAF1,CUUR0120SS47014,CUUR0120SAF114,CUUR0120SEFV,CUUR0350SAF1,CUUR0350SS47014,CUUR0350SEHF,CUUR0350SAR
0,2022,M09,296.808,123.305,140.733,117.432,104.002,122.624,143.880,120.718,122.205,123.290,143.971,125.356,112.321
1,2022,M08,296.171,121.508,162.745,116.483,100.224,122.056,160.519,120.943,121.009,122.504,155.459,123.913,112.424
2,2022,M07,296.276,119.860,180.223,115.843,99.280,121.499,177.177,119.824,120.489,121.735,179.455,124.084,111.795
3,2022,M06,296.311,118.936,195.148,116.036,100.838,119.713,187.678,118.246,119.704,120.211,198.044,120.047,111.612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,2018,M04,250.546,100.936,108.487,102.138,108.086,101.181,107.625,101.689,101.204,100.391,112.187,99.544,100.604
54,2018,M03,249.554,101.088,101.852,101.901,102.564,100.458,102.154,101.097,100.706,100.235,105.258,99.886,101.236
55,2018,M02,248.991,100.774,102.917,101.306,103.427,100.518,104.881,101.604,100.593,100.212,107.059,100.418,101.243
56,2018,M01,247.867,100.819,101.917,100.921,100.954,100.405,102.853,100.285,100.360,100.451,104.347,100.378,100.325


## Question:  Are all the series you downloaded in a monthly frequency?

Yes, every data set that we downloaded was collected in a monthly frequency starting in the last month of 2017 up until the present month.


In [7]:
#Aggregating CPI into one Column
cpitable['Aggregate CPI'] = (.432*cpitable['CUUR0110SAF1'] + .317*cpitable['CUUR0110SS47014'] + .015*cpitable['CUUR0110SAH'] +.01*cpitable['CUUR0110SAA'] + .022*cpitable['CUUR0120SAF114'] + 
                             .066*cpitable['CUUR0120SEFV'] + .092*cpitable['CUUR0350SEHF'] + .046*cpitable['CUUR0350SAR'])

In [8]:
cpitable

,year,period,CUUR0000SA0,CUUR0110SAF1,CUUR0110SS47014,CUUR0110SAH,CUUR0110SAA,CUUR0120SAF1,CUUR0120SS47014,CUUR0120SAF114,CUUR0120SEFV,CUUR0350SAF1,CUUR0350SS47014,CUUR0350SEHF,CUUR0350SAR,Aggregate CPI
0,2022,M09,296.808,123.305,140.733,117.432,104.002,122.624,143.880,120.718,122.205,123.290,143.971,125.356,112.321,128.102465
1,2022,M08,296.171,121.508,162.745,116.483,100.224,122.056,160.519,120.943,121.009,122.504,155.459,123.913,112.424,134.049946
2,2022,M07,296.276,119.860,180.223,115.843,99.280,121.499,177.177,119.824,120.489,121.735,179.455,124.084,111.795,138.787356
3,2022,M06,296.311,118.936,195.148,116.036,100.838,119.713,187.678,118.246,119.704,120.211,198.044,120.047,111.612,142.671540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,2018,M04,250.546,100.936,108.487,102.138,108.086,101.181,107.625,101.689,101.204,100.391,112.187,99.544,100.604,103.310115
54,2018,M03,249.554,101.088,101.852,101.901,102.564,100.458,102.154,101.097,100.706,100.235,105.258,99.886,101.236,101.228353
55,2018,M02,248.991,100.774,102.917,101.306,103.427,100.518,104.881,101.604,100.593,100.212,107.059,100.418,101.243,101.482977
56,2018,M01,247.867,100.819,101.917,100.921,100.954,100.405,102.853,100.285,100.360,100.451,104.347,100.378,100.325,101.064608


## Inflation

In [27]:
#The function below will compute the inflation for 12 months. 

def inflation2(lag,varname):
    
    # This function computes the percent change in the level of prices (inflation) for the 
    # 12 months starting '12+lag' months ago and ending 'lag' months ago.
    
    # lag - an integer indicating the lag in months
    # varname - a string containing the variable name (CUUR0000SA0 is the default)
    
    # Step 0: Check inputs
    if type(lag) != int:
        return 'lag should be an integer'
    if lag>12 or lag<0:
        return 'lag should be an integer between 0 and 12, including.'
    if type(varname) != str:
        return 'variable name should be a string (or left out)'
    
    # Step 1: API call
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
    url = base_url + varname
    r = requests.get(url).json()
    
    # Step 2: Check whether the GET request was successful
    if r['status'] != 'REQUEST_SUCCEEDED':
        return r['message']
  
    
    # Step 3: Compute inflation
    new = float(r['Results']['series'][0]['data'][lag]['value'])
    old = float(r['Results']['series'][0]['data'][12+lag]['value'])
    
    change = (new/old - 1)*100
    print('Inflation in the period starting ', lag, 'months ago and ending', 12+lag, 'months ago was ',round(change,1),'%')
    return change

We will compare the inflation for our CPI vs the CPI for the entire United States using 4 different lags. 

In [35]:
inflation2(0,'CUUR0000SA0')
inflation2(3,'CUUR0000SA0')
inflation2(9,'CUUR0000SA0')
inflation2(12,'CUUR0000SA0')

Inflation in the period starting  0 months ago and ending 12 months ago was  8.3 %
Inflation in the period starting  3 months ago and ending 15 months ago was  8.6 %
Inflation in the period starting  9 months ago and ending 21 months ago was  6.8 %
Inflation in the period starting  12 months ago and ending 24 months ago was  5.3 %


5.251271554874992

To measure the inflation for our CPI derived from our basket, we needed to make some changes to the original function. Now we are not calling the BLS API anymore, and we are instead retrieving values from the last column of our table. So firstly, we deleted the part of the function that related to API calls. Then, we changed the new/old variables to contain (.iat) functions which take values directly from our table. From there, we used the coordinates for our table to be 12 months behind for the row, and at a fixed value of 15 for our column. We left the built in error tests as they were. 

In [50]:
def inflation3 (lag,varname):
    
    if type(lag) != int:
        return 'lag should be an integer'
    if lag>12 or lag<0:
        return 'lag should be an integer between 0 and 12, including.'
    if type(varname) != str:
        return 'variable name should be a string (or left out)'
    
    new = float(cpitable.iat[lag,15])
    old = float(cpitable.iat[(lag+12),15])
    
    change = (new/old - 1)*100
    print('Inflation in the period starting ', lag, 'months ago and ending', 12+lag, 'months ago was ',round(change,1),'%')
    return change

In [51]:
Inflation = cpitable['Aggregate CPI']   #Assigning our column as a single variable to be compatible with the function.

In [53]:
inflation3(0,'Inflation')
inflation3(3,'Inflation')
inflation3(9,'Inflation')
inflation3(12,'Inflation')

Inflation in the period starting  0 months ago and ending 12 months ago was  17.7 %
Inflation in the period starting  3 months ago and ending 15 months ago was  23.0 %
Inflation in the period starting  9 months ago and ending 21 months ago was  22.4 %
Inflation in the period starting  12 months ago and ending 24 months ago was  14.9 %


14.868627315688032

## Inflation Analysis

The inflation we recieved for our aggregate CPI was much larger in all of our tested lags than the general CPI of the United States. This means that the price of items that college students typically use are growing faster than the average price of any good in the United States. This could also be partly because gas and food are heavily weighted and are very volatile with inflation.